In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
brownfields = pd.read_excel("/content/drive/MyDrive/MIT/re-powering-screening-dataset-2022.xlsx", sheet_name="RE-Powering Sites ")
brownfields.head

<bound method NDFrame.head of         Cross-Reference Number  EPA Region State      Program       Site ID  \
0                         2164           1    ME          AML  MED055715775   
1                         2222           1    VT          AML  VTN000105222   
2                         2490           1    CT  BROWNFIELDS        238836   
3                         2630           1    CT  BROWNFIELDS        220082   
4                         2850           1    CT  BROWNFIELDS         93605   
...                        ...         ...   ...          ...           ...   
190971                    1871          10    WA    SUPERFUND  WAD988519708   
190972                    1872          10    WA    SUPERFUND  WAD988475158   
190973                    1873          10    WA    SUPERFUND  WAD009487513   
190974                    1874          10    WA    SUPERFUND  WAD009248295   
190975                    1875          10    WA    SUPERFUND  WAD040187890   

                     

In [ ]:
brownfields.columns

Index(['Cross-Reference Number', 'EPA Region', 'State', 'Program', 'Site ID',
       'Site Information ', 'Site Name', 'Address', 'City', 'County',
       'Zip Code', 'Acreage (Acres)', 'Known Landfill',
       'Known Abandoned Mine Land', 'RE-Powering Profile',
       'Estimated PV Capacity (MW)', 'Utility Scale PV',
       'Distributed Scale PV', 'Off-grid PV', 'No Acreage PV',
       'Maximum Annual GHI (kWh/m2/day)',
       'Estimated Wind Energy Capacity (MW)', 'Utility Scale Wind',
       'Community Scale Wind', 'Facility Scale Wind', 'No Acreage Wind',
       'Wind speed at 40 m (m/s)', 'Wind speed at 80 m (m/s)',
       'Wind speed at 100 m (m/s)', 'Biopower Facility Potential ',
       'Biorefinery Facility Potential ',
       'Cumulative Biopower  Residues (metric tons / year within 50 miles)',
       'Cumulative Biorefinery  Residues (metric tons / year within 50 miles)',
       'Crop Residues (metric tons / year within 50 miles)',
       'Forest Residues (metric tons / year

In [ ]:
brownfields.shape

(190976, 50)

In [ ]:
brownfields['Cross-Reference Number'].unique().shape

(190976,)

In [ ]:
# Solar PV

utility_solar_pv = brownfields[brownfields['Utility Scale PV'] == 'Y']
distributed_scale_pv = brownfields[brownfields['Distributed Scale PV'] == 'Y']
off_grid_pv = brownfields[brownfields['Off-grid\xa0PV'] == 'Y']
no_acreage_pv = brownfields[brownfields['No Acreage PV'] == 'Y']
all_solar_pv = brownfields[(brownfields['Utility Scale PV'] == 'Y') | (brownfields['Distributed Scale PV'] == 'Y') | (brownfields['Off-grid\xa0PV'] == 'Y') | (brownfields['No Acreage PV'] == 'Y')]

In [ ]:
all_solar_pv.shape

(190976, 50)

In [ ]:
# Wind Capacity

utility_scale_wind = brownfields[brownfields['Utility Scale Wind'] == 'Y']
community_scale_wind = brownfields[brownfields['Community Scale Wind'] == 'Y']
facility_scale_wind = brownfields[brownfields['Facility Scale Wind'] == 'Y']
unknown_acreage_wind = brownfields[brownfields['No Acreage Wind'] == 'Y']
all_wind = brownfields[(brownfields['Utility Scale Wind'] == 'Y') | (brownfields['Community Scale Wind'] == 'Y') | (brownfields['Facility Scale Wind'] == 'Y') | (brownfields['No Acreage Wind'] == 'Y')]

In [ ]:
all_wind.shape

(111764, 50)

In [ ]:
all_biopower = brownfields[brownfields['Biopower Facility Potential '] == 'Y']
all_biorefinery = brownfields[brownfields['Biorefinery Facility Potential '] == 'Y']
all_biomass = brownfields[(brownfields['Biopower Facility Potential '] == 'Y') | (brownfields['Biorefinery Facility Potential '] == 'Y')]

In [ ]:
all_biomass.shape

(19071, 50)

In [ ]:
all_thermal = brownfields[brownfields['Geothermal Heat Pump Potential'] == 'Y']
all_thermal.shape

(190956, 50)

In [ ]:
common_sites_solar_wind = all_solar_pv.merge(all_wind, how='inner')
common_sites_solar_wind.shape

(111764, 50)

In [ ]:
# common_sites_solar_wind_bio = common_sites_solar_wind[(brownfields['Biopower Facility Potential '] == 'Y') | (brownfields['Biorefinery Facility Potential '] == 'Y')]
common_sites_solar_wind_bio = common_sites_solar_wind.merge(all_biomass, how='inner')
common_sites_solar_wind_bio.shape

(9357, 50)

In [ ]:
common_sites_solar_wind_bio_thermal = common_sites_solar_wind_bio.merge(all_thermal, how='inner')
common_sites_solar_wind_bio_thermal.shape

(9357, 50)

In [ ]:
cond = all_solar_pv['Cross-Reference Number'].isin(common_sites_solar_wind_bio_thermal['Cross-Reference Number'])
only_solar_sites = all_solar_pv.drop(all_solar_pv[cond].index)
only_solar_sites.shape

(181619, 50)

In [ ]:
cond = all_wind['Cross-Reference Number'].isin(common_sites_solar_wind_bio_thermal['Cross-Reference Number'])
only_wind_sites = all_wind.drop(all_wind[cond].index)
only_wind_sites.shape

(102407, 50)

In [ ]:
cond = all_biomass['Cross-Reference Number'].isin(common_sites_solar_wind_bio_thermal['Cross-Reference Number'])
only_biomass_sites = all_biomass.drop(all_biomass[cond].index)
only_biomass_sites.shape

(9714, 50)

In [ ]:
cond = all_thermal['Cross-Reference Number'].isin(common_sites_solar_wind_bio_thermal['Cross-Reference Number'])
only_thermal_sites = all_thermal.drop(all_thermal[cond].index)
only_thermal_sites.shape

(181599, 50)

In [ ]:
def type_func(s):
  if((s['Estimated PV Capacity (MW)'] > 10) & (s['Estimated PV Capacity (MW)'] > s['Estimated Wind Energy Capacity (MW)'])):
    return 'Solar'
  elif ((s['Estimated Wind Energy Capacity (MW)'] > 10) & (s['Estimated PV Capacity (MW)'] < s['Estimated Wind Energy Capacity (MW)'])):
    return 'Wind'
  else:
    return 'Biomass'



In [ ]:
common_sites_solar_wind_bio_thermal['type'] = common_sites_solar_wind_bio_thermal.apply(type_func, axis=1)

common_sites_wind = common_sites_solar_wind_bio_thermal[common_sites_solar_wind_bio_thermal['type']=='Wind']
print(common_sites_wind.shape)

common_sites_solar = common_sites_solar_wind_bio_thermal[common_sites_solar_wind_bio_thermal['type']=='Solar']
print(common_sites_solar.shape)

common_sites_bio = common_sites_solar_wind_bio_thermal[common_sites_solar_wind_bio_thermal['type']=='Biomass']
print(common_sites_bio.shape)

(73, 51)
(2955, 51)
(6329, 51)


In [ ]:
only_solar_sites['type'] = 'Solar'
only_wind_sites['type'] = 'Wind'
only_biomass_sites['type'] = 'Biopower'
only_thermal_sites['type'] = 'Thermal'

In [ ]:
only_solar = pd.concat([only_solar_sites, common_sites_solar])
only_solar.shape

(184574, 51)

In [ ]:
only_wind = pd.concat([only_wind_sites, common_sites_wind])
only_wind.shape

(102480, 51)

In [ ]:
only_bio = pd.concat([only_biomass_sites, common_sites_bio])
only_bio.shape

(16043, 51)

In [ ]:
only_thermal = only_thermal_sites
only_thermal.shape

(181599, 51)

In [ ]:
only_solar_capacity = only_solar['Estimated PV Capacity (MW)'].sum()
only_solar_capacity

518908.8276389073

In [ ]:
only_wind_capacity = only_wind['Estimated Wind Energy Capacity (MW)'].sum()
only_wind_capacity

123280.10873450006

In [ ]:
only_bio_capacity = len(only_bio) * 10
only_bio_capacity

160430

In [ ]:
total_capacity = only_solar_capacity + only_wind_capacity + only_bio_capacity
total_capacity

802618.9363734074

In [ ]:
statewise_bf = brownfields.groupby('State').size().reset_index(name='no_of_sites')
statewise_bf.sort_values(by=['no_of_sites'], ascending=False, inplace=True)
statewise_bf

In [ ]:
statewise_landfill = brownfields[brownfields['Known Landfill'] == 'Y']
statewise_landfill = statewise_landfill.groupby('State').size().reset_index(name='no_of_sites')
statewise_landfill.sort_values(by=['no_of_sites'], ascending=False, inplace=True)
statewise_landfill

In [ ]:
statewise_landmine = brownfields[brownfields['Known Abandoned Mine Land'] == 'Y']
statewise_landmine = statewise_landmine.groupby('State').size().reset_index(name='no_of_sites')
statewise_landmine.sort_values(by=['no_of_sites'], ascending=False, inplace=True)
statewise_landmine

In [ ]:
# coal_mines = pd.read_excel("/content/drive/MyDrive/MIT/Coal Mine Closures since 1999.xlsx", sheet_name="Sheet1")
# coal_plants = pd.read_excel("/content/drive/MyDrive/MIT/Coal Plant Closures since 2009.xlsx", sheet_name="Sheet1")
active_coal_plants = pd.read_excel("/content/drive/MyDrive/MIT/Coal Generators 2021.xlsx", sheet_name="Operable")

In [ ]:
statewise_active_coal_plants = active_coal_plants.groupby(['Plant Code', 'State']).agg({'Nameplate Capacity (MW)': 'sum'}).reset_index()

In [ ]:
statewise_coalplants = statewise_active_coal_plants.groupby('State').size().reset_index(name='no_of_plants')
statewise_coalplants.sort_values(by=['no_of_plants'], ascending=False, inplace=True)
statewise_coalplants

In [ ]:
statewise_coalplants_capacity = statewise_active_coal_plants.groupby('State').agg({'Nameplate Capacity (MW)': 'sum'}).reset_index()
statewise_coalplants_capacity.sort_values(by=['Nameplate Capacity (MW)'], ascending=False, inplace=True)
statewise_coalplants_capacity

In [ ]:
statewise_coalplants_capacity.to_excel('/content/drive/MyDrive/MIT/CoalPlants_Capacity.xlsx')

In [ ]:
ej = pd.read_csv("/content/drive/MyDrive/MIT/EJSCREEN_2021_USPR.csv")
ej.shape

(220333, 155)

In [ ]:
if 'SHAPE' not in ej:
  print(True)

True


In [ ]:
ej_req = ej[['OBJECTID', 'ST_ABBREV', 'STATE_NAME', 'P_PM25', 'P_CANCR', 'P_RESP', 'P_PTSDF', 'P_MINORPCT', 'P_UNEMPPCT', 'P_LNGISPCT']]
ej_req.shape

(220333, 10)

In [ ]:
pm25 = ej_req.sort_values('P_PM25',ascending = False).head(22033)
pm25.to_excel("/content/drive/MyDrive/MIT/PM25.xlsx")
pm25['STATE_NAME'].unique()

array(['California', 'Idaho', 'Montana', 'Oregon', 'Washington', 'Texas',
       'Oklahoma', 'Illinois'], dtype=object)

In [ ]:
cancer_risk = ej_req.sort_values('P_CANCR',ascending = False).head(22033)
cancer_risk.to_excel("/content/drive/MyDrive/MIT/P_CANCR.xlsx")
cancer_risk['STATE_NAME'].unique()

array(['Puerto Rico', 'Louisiana', 'Texas', 'Arkansas', 'Arizona',
       'Pennsylvania', 'Florida', 'Virginia', 'Tennessee', 'Illinois',
       'Washington', 'California', 'Missouri', 'Colorado', 'Georgia',
       'New Jersey', 'Michigan', 'New Mexico', 'West Virginia',
       'Delaware', 'New York', 'Utah', 'South Carolina', 'Oregon', 'Iowa',
       'Nebraska', 'Montana', 'Alaska', 'Mississippi', 'Minnesota',
       'Alabama', 'Maryland', 'Kentucky', 'Ohio', 'Hawaii',
       'Rhode Island', 'Oklahoma', 'North Carolina', 'Nevada', 'Indiana',
       'District of Columbia', 'Kansas'], dtype=object)

In [ ]:
resp_risk = ej_req.sort_values('P_RESP',ascending = False).head(22033)
resp_risk.to_excel("/content/drive/MyDrive/MIT/P_RESP.xlsx")
resp_risk['STATE_NAME'].unique()

array(['Washington', 'California', 'Montana', 'Kentucky',
       'North Carolina', 'Colorado', 'Minnesota', 'Louisiana', 'Texas',
       'Alaska', 'West Virginia', 'Oregon', 'Georgia', 'Hawaii',
       'Mississippi', 'Tennessee', 'Pennsylvania', 'Utah', 'Virginia',
       'New York', 'Wisconsin', 'Illinois', 'Florida', 'Ohio', 'Alabama',
       'Kansas', 'Massachusetts', 'South Carolina', 'Arkansas', 'Arizona',
       'Nevada', 'Idaho', 'Oklahoma', 'New Jersey', 'Missouri',
       'District of Columbia', 'Maryland', 'Maine', 'Indiana',
       'New Mexico'], dtype=object)

In [ ]:
hw_prox = ej_req.sort_values('P_PTSDF',ascending = False).head(22033)
hw_prox.to_excel("/content/drive/MyDrive/MIT/P_PTSDF.xlsx")
hw_prox['STATE_NAME'].unique()

array(['Massachusetts', 'Rhode Island', 'New York',
       'District of Columbia', 'California', 'Hawaii', 'New Jersey',
       'Washington', 'Illinois', 'Maryland', 'West Virginia',
       'Pennsylvania', 'Minnesota', 'Florida', 'Connecticut', 'Indiana',
       'Texas', 'Ohio', 'Oregon', 'Virginia', 'Michigan', 'Arizona',
       'Wisconsin', 'Maine', 'Missouri', 'Alabama', 'Louisiana',
       'Delaware', 'Kansas', 'Tennessee', 'Nebraska', 'North Carolina',
       'Kentucky', 'New Hampshire', 'Puerto Rico', 'South Carolina',
       'Colorado', 'Utah', 'New Mexico', 'Montana', 'Oklahoma', 'Nevada',
       'Vermont', 'Georgia'], dtype=object)

In [ ]:
minor_pct = ej_req.sort_values('P_MINORPCT',ascending = False).head(22033)
minor_pct.to_excel("/content/drive/MyDrive/MIT/P_MINORPCT.xlsx")
minor_pct['STATE_NAME'].unique()

array(['Mississippi', 'Michigan', 'Missouri', 'Montana', 'Nevada',
       'Maryland', 'Louisiana', 'Massachusetts', 'New Jersey', 'New York',
       'New Mexico', 'California', 'Florida', 'District of Columbia',
       'Connecticut', 'Delaware', 'Arkansas', 'Arizona', 'Illinois',
       'Kentucky', 'Indiana', 'Kansas', 'Georgia', 'Hawaii',
       'Puerto Rico', 'Tennessee', 'South Carolina', 'Pennsylvania',
       'Rhode Island', 'Texas', 'North Carolina', 'Ohio', 'North Dakota',
       'Oklahoma', 'Wisconsin', 'Virginia', 'Washington', 'Alabama',
       'Alaska', 'Utah', 'South Dakota', 'Colorado', 'Nebraska',
       'West Virginia', 'Minnesota', 'Idaho', 'Oregon', 'Iowa', 'Wyoming'],
      dtype=object)

In [ ]:
umemp_pct = ej_req.sort_values('P_UNEMPPCT',ascending = False).head(22033)
umemp_pct.to_excel("/content/drive/MyDrive/MIT/P_UNEMPPCT.xlsx")
umemp_pct['STATE_NAME'].unique()

array(['Puerto Rico', 'Illinois', 'Kentucky', 'Tennessee', 'New Jersey',
       'Virginia', 'Florida', 'Ohio', 'New York', 'South Carolina',
       'Georgia', 'Maryland', 'California', 'Arkansas', 'North Carolina',
       'Arizona', 'Louisiana', 'Nebraska', 'Oklahoma', 'Indiana',
       'District of Columbia', 'Michigan', 'Texas', 'Pennsylvania',
       'Nevada', 'Mississippi', 'Idaho', 'West Virginia', 'Iowa',
       'New Mexico', 'Alabama', 'Missouri', 'Massachusetts', 'Oregon',
       'Rhode Island', 'Washington', 'Connecticut', 'Alaska', 'Colorado',
       'Wisconsin', 'Kansas', 'Montana', 'South Dakota', 'Delaware',
       'Minnesota', 'Maine', 'Utah', 'Hawaii', 'North Dakota',
       'New Hampshire', 'Vermont', 'Wyoming'], dtype=object)

In [ ]:
limeng_pct = ej_req.sort_values('P_LNGISPCT',ascending = False).head(22033)
limeng_pct.to_excel("/content/drive/MyDrive/MIT/P_LNGISPCT.xlsx")
limeng_pct['STATE_NAME'].unique()

array(['Kentucky', 'Puerto Rico', 'Florida', 'Texas', 'Maryland',
       'California', 'New York', 'Missouri', 'New Jersey', 'Hawaii',
       'Virginia', 'Massachusetts', 'Minnesota', 'Georgia', 'Connecticut',
       'Nevada', 'Arizona', 'Illinois', 'Pennsylvania', 'North Carolina',
       'Tennessee', 'Kansas', 'Indiana', 'New Mexico', 'Oregon', 'Ohio',
       'Rhode Island', 'Washington', 'Wisconsin', 'Oklahoma', 'Arkansas',
       'Nebraska', 'Michigan', 'District of Columbia', 'Alabama', 'Iowa',
       'Colorado', 'West Virginia', 'Idaho', 'South Carolina',
       'Louisiana', 'Alaska', 'Mississippi', 'Delaware', 'Utah',
       'Wyoming', 'New Hampshire', 'Maine', 'South Dakota', 'Montana',
       'North Dakota', 'Vermont'], dtype=object)

In [ ]:
ej_req.fillna(0, inplace=True)
ej_req = ej_req.groupby(['STATE_NAME', 'ST_ABBREV']).mean().reset_index()
ej_req

,STATE_NAME,ST_ABBREV,P_PM25,P_CANCR,P_RESP,P_PTSDF,P_MINORPCT,P_UNEMPPCT,P_LNGISPCT
0,Alabama,AL,83.801256,99.430460,99.458015,97.786056,100.000000,99.966616,98.144579
1,Alaska,AK,0.000000,99.636868,99.898622,75.170352,98.051079,99.891401,97.601952
2,Arizona,AZ,70.375954,99.992282,99.458015,98.180429,100.000000,100.000000,99.293883
3,Arkansas,AR,84.929690,99.992282,99.458015,89.130101,100.000000,100.000000,98.603214
4,California,CA,100.000000,99.985074,99.997968,99.933730,100.000000,100.000000,100.000000
5,Colorado,CO,68.446051,99.985074,99.970196,95.066333,97.173391,99.878848,98.014740
6,Connecticut,CT,40.820861,83.776762,47.621596,98.942747,100.000000,99.899295,99.513689
7,Delaware,DE,70.953020,99.834080,81.591503,97.507372,100.000000,99.776367,96.535217
8,District of Columbia,DC,52.612120,97.949929,95.252022,99.993713,100.000000,99.994302,98.350965
9,Florida,FL,45.514619,99.985074,99.642520,99.176854,100.000000,100.000000,100.000000
